导入包

In [4]:
import pandas as pd
import numpy as np

读入数据

## 数据重塑和轴向旋转

（1）层次化索引

层次化索引是pandas的一项重要功能，它能使我们在一个轴上拥有多个索引,因此可以低维度的形式处理高维度的数据(如以外层索引的方式选择数据子集，

以内层索引的方式选择数据)。实质上，单级索引对应Index对象,多级索引对应MultiIndex对象

Series的层次化索引

In [13]:
s=pd.Series(np.arange(1,10),
            index=[['a','a','a','b','b','c','c','d','d'],
                   [1,2,3,1,2,3,1,2,3]])
s

a  1    1
   2    2
   3    3
b  1    4
   2    5
c  3    6
   1    7
d  2    8
   3    9
dtype: int32

In [10]:
s.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 1, 2, 0, 1, 2]])

In [11]:
s['b']

1    4
2    5
dtype: int32

In [12]:
s['b':'c']

b  1    4
   2    5
c  3    6
   1    7
dtype: int32

In [8]:
s[:,2]

a    2
b    5
d    8
dtype: int32

In [14]:
s['c',3]

6

通过unstack(反堆叠)方法可以将Series变成一个DataFrame
![访问方式](./png/层次化索引.png)

In [15]:
s.unstack(level=1) # level为0或1，默认为1 表示外层为行标签，内层为列标签；否则相反

,1,2,3
a,1.0,2.0,3.0
b,4.0,5.0,NaN
c,7.0,NaN,6.0
d,NaN,8.0,9.0


In [16]:
s.unstack().stack()

a  1    1.0
   2    2.0
   3    3.0
b  1    4.0
   2    5.0
c  1    7.0
   3    6.0
d  2    8.0
   3    9.0
dtype: float64

DataFrame的层次化索引

对于DataFrame来说，行和列都能够进行层次化索引。

In [17]:
#给定一组比赛的数据，包括少年组Junior和青年组Youth,需要描述的信息包括：不同组别的不同人，在不同时间段的不同场次的比赛得分，便可以用
#层次化索引表示
Gamedata=pd.DataFrame(np.array([[31,27,24,60],[26,28,13,29],
                                [27,17,12,27],[29,9,5,18],
                                [31,12,4,70],[45,11,12,12]])
                      ,index=[['Morning','Morning','Morning','Afternoon','Afternoon','Afternoon'],
                              [1,2,3,1,2,3]],
                      columns=[['Junior','Junior','Youth','Youth'],
                               ['Zhang','Wang','Li','Zhao']])
Gamedata

Junior      Youth     
             Zhang Wang    Li Zhao
Morning   1     31   27    24   60
          2     26   28    13   29
          3     27   17    12   27
Afternoon 1     29    9     5   18
          2     31   12     4   70
          3     45   11    12   12

In [18]:
Gamedata.columns.names=['age','name']
Gamedata

age         Junior      Youth     
name         Zhang Wang    Li Zhao
Morning   1     31   27    24   60
          2     26   28    13   29
          3     27   17    12   27
Afternoon 1     29    9     5   18
          2     31   12     4   70
          3     45   11    12   12

In [22]:
Gamedata.index.names=['time','sequence']
Gamedata

age                Junior      Youth     
name                Zhang Wang    Li Zhao
time      sequence                       
Morning   1            31   27    24   60
          2            26   28    13   29
          3            27   17    12   27
Afternoon 1            29    9     5   18
          2            31   12     4   70
          3            45   11    12   12

In [19]:
Gamedata['Junior']

name         Zhang  Wang
Morning   1     31    27
          2     26    28
          3     27    17
Afternoon 1     29     9
          2     31    12
          3     45    11

In [23]:
Gamedata.loc['Morning']

age      Junior      Youth     
name      Zhang Wang    Li Zhao
sequence                       
1            31   27    24   60
2            26   28    13   29
3            27   17    12   27

In [24]:
Gamedata.swaplevel('time','sequence')

age                Junior      Youth     
name                Zhang Wang    Li Zhao
sequence time                            
1        Morning       31   27    24   60
2        Morning       26   28    13   29
3        Morning       27   17    12   27
1        Afternoon     29    9     5   18
2        Afternoon     31   12     4   70
3        Afternoon     45   11    12   12

In [27]:
Gamedata.swaplevel('age','name',axis=1)

name                Zhang   Wang    Li  Zhao
age                Junior Junior Youth Youth
time      sequence                          
Morning   1            31     27    24    60
          2            26     28    13    29
          3            27     17    12    27
Afternoon 1            29      9     5    18
          2            31     12     4    70
          3            45     11    12    12

了解了层次化索引的基本知识之后，我们试着将电影数据也处理成一种多层索引的结构。

In [5]:
df=pd.read_excel("dfilm.xls ")
df

,名字,投票人数,类型,产地,上映时间,时长,年代,评分,首映地点
0,肖申克的救赎,692795,剧情/犯罪,美国,1994-09-10 00:00:00,142,1994,9.6,多伦多电影节
1,控方证人,42995,剧情/悬疑/犯罪,美国,1957-12-17 00:00:00,116,1957,9.5,美国
2,美丽人生,327855,剧情/喜剧/爱情,意大利,1997-12-20 00:00:00,116,1997,9.5,意大利
3,阿甘正传,580897,剧情/爱情,美国,1994-06-23 00:00:00,142,1994,9.4,洛杉矶首映
4,霸王别姬,478523,剧情/爱情/同性,中国大陆,1993-01-01 00:00:00,171,1993,9.4,香港
5,泰坦尼克号,157074,剧情/爱情/灾难,美国,2012-04-10 00:00:00,194,2012,9.4,中国大陆
6,辛德勒的名单,306904,剧情/历史/战争,美国,1993-11-30 00:00:00,195,1993,9.4,华盛顿首映
7,新世纪福音战士剧场版：Air/真心为你 新世紀エヴァンゲリオン劇場版 Ai,24355,剧情/动作/科幻/动画/奇幻,日本,1997-07-19 00:00:00,87,1997,9.4,日本
8,银魂完结篇：直到永远的万事屋 劇場版 銀魂 完結篇 万事屋よ,21513,剧情/动画,日本,2013-07-06 00:00:00,110,2013,9.4,日本
9,这个杀手不太冷,662552,剧情/动作/犯罪,法国,1994-09-14 00:00:00,133,1994,9.4,法国


In [7]:
df.index

RangeIndex(start=0, stop=38736, step=1)

把产地和年代同时设成索引，产地是外层索引，年代为内层索引

set_index可以把列变成索引

resset_index是把索引变成列

In [8]:
df=df.set_index(['产地','年代'])
df

名字    投票人数  \
产地   年代                                                                
美国   1994                                             肖申克的救赎  692795   
     1957                                               控方证人   42995   
意大利  1997                                              美丽人生   327855   
美国   1994                                               阿甘正传  580897   
中国大陆 1993                                               霸王别姬  478523   
美国   2012                                             泰坦尼克号   157074   
     1993                                             辛德勒的名单  306904   
日本   1997              新世纪福音战士剧场版：Air/真心为你 新世紀エヴァンゲリオン劇場版 Ai   24355   
     2013                     银魂完结篇：直到永远的万事屋 劇場版 銀魂 完結篇 万事屋よ   21513   
法国   1994                                           这个杀手不太冷   662552   
意大利  2003                                              灿烂人生    16807   
美国   2016                                              疯狂动物城  284652   
日本   2009                           福音战士新剧场版：破 ヱヴァンゲリヲン新劇場版：   32524   
美国   2009                                                海豚湾  159302   
日本   2008                                       回忆积木小屋 つみきのい   93384   
美国   2008                                            机器人总动员   421734   
     1957                                               十二怒汉  134949   
     2008                                            旅行到宇宙边缘   10044   
英国   2001                                                父与女   53358   
美国   2009                                            暴力云与送子鹳   75567   
     1931                                               城市之光   31105   
中国大陆 1961                                               大闹天宫   74881   
美国   2010                                               盗梦空间  642134   
法国   2004                                             放牛班的春天  370585   
意大利  1998                                              海上钢琴师  501153   
法国   2009                                                 家园   32717   
美国   1972                                                 教父  280871   
     1939                                               乱世佳人  226131   
日本   2001                                       千与千寻 千と千尋の神隠  525505   
中国大陆 2015                                               穹顶之下   51113   
...                                                      ...     ...   
美国   1990                                                 洛奇    3211   
英国   1990                                             孟菲斯美女号    1627   
     1990                                            末路英雄半世情     216   
美国   1990                                              活死人之夜    1326   
中国香港 1989                                   省港旗兵3：逃出香港 省港旗兵第    1399   
美国   1989                                               危险之至      85   
意大利  1989                                               拯救首相     111   
日本   1989                                                二二六     568   
波兰   2014                                               华沙谍战     719   
中国大陆 1989                                              地狱·天堂     171   
韩国   1989                                            达摩为何东渡？     291   
美国   1988                                                天伦乐      99   
俄罗斯  1988                        青春禁忌游戏 Дорогая Елена Сергее     118   
英国   1987                                             安迪·沃霍尔      40   
美国   1987                                               青春传奇     175   
     1987                                              我要求审判     102   
     1987                                              跷家的一夜      82   
中国台湾 1987                                         黑皮与白牙 黑皮與白     106   
美国   1987                                              零下的激情     199   
中国大陆 1986                                           T省的八四、八五     380   
美国   1986                                               离别秋波     240   
中国大陆 1986                                            失踪的女中学生     101   
瑞典   1986                                            喧闹村的孩子们      36   
中国大陆 1986                 

每一个索引都是一个元组

In [9]:
df.index[0]

('美国', 1994)

获取所有的美国电影，由于产地信息已经变成了索引，因此要是用.loc方法

In [10]:
df.loc['美国'] #获得以年代为索引的数据

,名字,投票人数,类型,上映时间,时长,评分,首映地点
年代,,,,,,,
1994,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.6,多伦多电影节
1957,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.5,美国
1994,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映
2012,泰坦尼克号,157074,剧情/爱情/灾难,2012-04-10 00:00:00,194,9.4,中国大陆
1993,辛德勒的名单,306904,剧情/历史/战争,1993-11-30 00:00:00,195,9.4,华盛顿首映
2016,疯狂动物城,284652,喜剧/动作/动画/冒险,2016-03-04 00:00:00,109,9.3,中国大陆/美国
2009,海豚湾,159302,纪录片,2009-07-31 00:00:00,92,9.3,美国
2008,机器人总动员,421734,喜剧/爱情/科幻/动画/冒险,2008-06-27 00:00:00,98,9.3,美国
1957,十二怒汉,134949,剧情,1957-04-01 00:00:00,96,9.3,美国


In [11]:
df.loc['中国大陆']

,名字,投票人数,类型,上映时间,时长,评分,首映地点
年代,,,,,,,
1993,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.4,香港
1961,大闹天宫,74881,动画/奇幻,1905-05-14 00:00:00,114,9.2,上集
2015,穹顶之下,51113,纪录片,2015-02-28 00:00:00,104,9.2,中国大陆
1982,茶馆,10678,剧情/历史,1905-06-04 00:00:00,118,9.2,美国
1988,山水情,10781,动画/短片,1905-06-10 00:00:00,19,9.2,美国
2016,“人权卫士”的人权纪录,109,纪录片,2016-03-13 00:00:00,45,5.6,中国大陆
2014,0.5,2825,爱情,2014-09-12 00:00:00,85,4.4,中国大陆
2000,鬼子来了,180738,剧情/战争,2000-05-12 00:00:00,139,9.1,戛纳电影节
2008,11度青春之《李雷和韩梅,9583,短片,2010-10-21 00:00:00,60,5.7,美国


取消层次化索引

In [12]:
df=df.reset_index()
df

,产地,年代,名字,投票人数,类型,上映时间,时长,评分,首映地点
0,美国,1994,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.6,多伦多电影节
1,美国,1957,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.5,美国
2,意大利,1997,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.5,意大利
3,美国,1994,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映
4,中国大陆,1993,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.4,香港
5,美国,2012,泰坦尼克号,157074,剧情/爱情/灾难,2012-04-10 00:00:00,194,9.4,中国大陆
6,美国,1993,辛德勒的名单,306904,剧情/历史/战争,1993-11-30 00:00:00,195,9.4,华盛顿首映
7,日本,1997,新世纪福音战士剧场版：Air/真心为你 新世紀エヴァンゲリオン劇場版 Ai,24355,剧情/动作/科幻/动画/奇幻,1997-07-19 00:00:00,87,9.4,日本
8,日本,2013,银魂完结篇：直到永远的万事屋 劇場版 銀魂 完結篇 万事屋よ,21513,剧情/动画,2013-07-06 00:00:00,110,9.4,日本
9,法国,1994,这个杀手不太冷,662552,剧情/动作/犯罪,1994-09-14 00:00:00,133,9.4,法国


(2) 数据旋转

行列转化：以前5部电影为例



In [35]:
data=df[:5]
data

名字    投票人数        类型                 上映时间   时长   评分    首映地点
产地   年代                                                                   
美国   1994  肖申克的救赎  692795     剧情/犯罪  1994-09-10 00:00:00  142  9.6  多伦多电影节
     1957    控方证人   42995  剧情/悬疑/犯罪  1957-12-17 00:00:00  116  9.5      美国
意大利  1997   美丽人生   327855  剧情/喜剧/爱情  1997-12-20 00:00:00  116  9.5     意大利
美国   1994    阿甘正传  580897     剧情/爱情  1994-06-23 00:00:00  142  9.4   洛杉矶首映
中国大陆 1993    霸王别姬  478523  剧情/爱情/同性  1993-01-01 00:00:00  171  9.4      香港

In [36]:
data.T

产地                     美国                                       意大利  \
年代                   1994                 1957                 1997   
名字                 肖申克的救赎                 控方证人                美丽人生    
投票人数               692795                42995               327855   
类型                  剧情/犯罪             剧情/悬疑/犯罪             剧情/喜剧/爱情   
上映时间  1994-09-10 00:00:00  1957-12-17 00:00:00  1997-12-20 00:00:00   
时长                    142                  116                  116   
评分                    9.6                  9.5                  9.5   
首映地点               多伦多电影节                   美国                  意大利   

产地                     美国                 中国大陆  
年代                   1994                 1993  
名字                   阿甘正传                 霸王别姬  
投票人数               580897               478523  
类型                  剧情/爱情             剧情/爱情/同性  
上映时间  1994-06-23 00:00:00  1993-01-01 00:00:00  
时长                    142                  171  
评分                    9.4                  9.4  
首映地点                洛杉矶首映                   香港

In [37]:
data.stack()

产地    年代        
美国    1994  名字                   肖申克的救赎
            投票人数                 692795
            类型                    剧情/犯罪
            上映时间    1994-09-10 00:00:00
            时长                      142
            评分                      9.6
            首映地点                 多伦多电影节
      1957  名字                     控方证人
            投票人数                  42995
            类型                 剧情/悬疑/犯罪
            上映时间    1957-12-17 00:00:00
            时长                      116
            评分                      9.5
            首映地点                     美国
意大利   1997  名字                    美丽人生 
            投票人数                 327855
            类型                 剧情/喜剧/爱情
            上映时间    1997-12-20 00:00:00
            时长                      116
            评分                      9.5
            首映地点                    意大利
美国    1994  名字                     阿甘正传
            投票人数                 580897
            类型                    剧情/爱情
            上映时间    199

In [164]:
data.stack().unstack()

,产地,年代,名字,投票人数,类型,上映时间,时长,评分,首映地点
0,美国,1994,肖申克的救赎,692795,剧情/犯罪,1994-09-10 00:00:00,142,9.6,多伦多电影节
1,美国,1957,控方证人,42995,剧情/悬疑/犯罪,1957-12-17 00:00:00,116,9.5,美国
2,意大利,1997,美丽人生,327855,剧情/喜剧/爱情,1997-12-20 00:00:00,116,9.5,意大利
3,美国,1994,阿甘正传,580897,剧情/爱情,1994-06-23 00:00:00,142,9.4,洛杉矶首映
4,中国大陆,1993,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.4,香港


## 数据分组、分组运算
GroupBy技术：实现数据的分组，和分组运算，作用类似于数据透视表

![访问方式](./png/分组计算.png)

按照电影的产地进行分组,先定义一个分组变量group

In [38]:
df=pd.read_excel("dfilm.xls ")
group=df.groupby(df['产地'])
group

可以计算分组后的各个统计量

In [41]:
group.mean()

,投票人数,评分
产地,,
中国台湾,8474.864078,7.066667
中国大陆,10898.293793,6.064703
中国香港,8164.554348,6.474053
丹麦,1993.858586,7.245960
俄罗斯,1019.134172,7.554507
其他,1590.686979,7.237448
加拿大,1915.304288,6.733610
印度,3210.843137,6.872269
墨西哥,1173.218487,7.087395


In [42]:
group.sum()

,投票人数,评分
产地,,
中国台湾,5237466,4367.2
中国大陆,41435313,23058.0
中国香港,23285309,18464.0
丹麦,394784,1434.7
俄罗斯,486127,3603.5
其他,3054119,13895.9
加拿大,1384765,4868.4
印度,1146271,2453.4
墨西哥,139613,843.4


In [43]:
df['评分']

0        9.6
1        9.5
2        9.5
3        9.4
4        9.4
5        9.4
6        9.4
7        9.4
8        9.4
9        9.4
10       9.3
11       9.3
12       9.3
13       9.3
14       9.3
15       9.3
16       9.3
17       9.3
18       9.2
19       9.2
20       9.2
21       9.2
22       9.2
23       9.2
24       9.2
25       9.2
26       9.2
27       9.2
28       9.2
29       9.2
        ... 
38706    7.5
38707    8.4
38708    8.1
38709    6.8
38710    6.1
38711    8.0
38712    7.7
38713    6.9
38714    7.0
38715    6.2
38716    8.1
38717    8.5
38718    9.1
38719    8.1
38720    8.2
38721    8.2
38722    7.8
38723    8.0
38724    7.4
38725    8.7
38726    8.2
38727    7.4
38728    8.7
38729    8.1
38730    8.1
38731    8.6
38732    7.6
38733    7.1
38734    8.0
38735    6.6
Name: 评分, Length: 38736, dtype: float64

计算每年的平均评分

In [44]:
df['评分'].groupby(df['年代']).mean() 

年代
1888    7.950000
1890    4.800000
1892    7.500000
1894    6.633333
1895    7.575000
1896    7.037500
1897    6.633333
1898    7.450000
1899    6.900000
1900    7.228571
1901    7.250000
1902    7.483333
1903    6.968750
1904    7.212500
1905    6.820000
1906    7.342857
1907    7.020000
1908    7.150000
1909    7.560000
1910    6.940000
1911    7.375000
1912    7.920000
1913    6.862500
1914    6.473077
1915    7.260000
1916    7.758333
1917    7.075000
1918    7.200000
1919    7.490000
1920    7.492857
          ...   
1987    7.284548
1988    7.265729
1989    7.210687
1990    7.149109
1991    7.154634
1992    7.223678
1993    7.195692
1994    7.262348
1995    7.289366
1996    7.258801
1997    7.330168
1998    7.238302
1999    7.188000
2000    7.134063
2001    7.110145
2002    7.069778
2003    7.137750
2004    7.032130
2005    7.030008
2006    6.914719
2007    6.875102
2008    6.911303
2009    6.750000
2010    6.770361
2011    6.577010
2012    6.458374
2013    6.392604
2014    6.2

In [45]:
#df.groupby(df['年代'])['评分'].mean()
df.groupby(df.年代)['评分'].mean()
#也可以

年代
1888    7.950000
1890    4.800000
1892    7.500000
1894    6.633333
1895    7.575000
1896    7.037500
1897    6.633333
1898    7.450000
1899    6.900000
1900    7.228571
1901    7.250000
1902    7.483333
1903    6.968750
1904    7.212500
1905    6.820000
1906    7.342857
1907    7.020000
1908    7.150000
1909    7.560000
1910    6.940000
1911    7.375000
1912    7.920000
1913    6.862500
1914    6.473077
1915    7.260000
1916    7.758333
1917    7.075000
1918    7.200000
1919    7.490000
1920    7.492857
          ...   
1987    7.284548
1988    7.265729
1989    7.210687
1990    7.149109
1991    7.154634
1992    7.223678
1993    7.195692
1994    7.262348
1995    7.289366
1996    7.258801
1997    7.330168
1998    7.238302
1999    7.188000
2000    7.134063
2001    7.110145
2002    7.069778
2003    7.137750
2004    7.032130
2005    7.030008
2006    6.914719
2007    6.875102
2008    6.911303
2009    6.750000
2010    6.770361
2011    6.577010
2012    6.458374
2013    6.392604
2014    6.2

In [174]:
df['年代']=df['年代'].astype(str)#只对数值变量计算

In [176]:
type(df['年代'].ix[0])

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


str

In [47]:
df.groupby(df['产地']).mean()

,投票人数,评分
产地,,
中国台湾,8474.864078,7.066667
中国大陆,10898.293793,6.064703
中国香港,8164.554348,6.474053
丹麦,1993.858586,7.245960
俄罗斯,1019.134172,7.554507
其他,1590.686979,7.237448
加拿大,1915.304288,6.733610
印度,3210.843137,6.872269
墨西哥,1173.218487,7.087395


我们也可以传入多个分组变量

In [177]:
df.groupby([df.产地,df.年代]).mean()

投票人数          时长        评分
产地   年代                                      
中国台湾 1963    121.000000  113.000000  6.400000
     1965    153.666667  105.000000  6.800000
     1966     51.000000   60.000000  7.900000
     1967   4444.000000  112.000000  8.000000
     1968     89.000000   83.000000  7.400000
     1969     91.000000   86.333333  7.400000
     1970    992.166667  108.666667  7.266667
     1971     51.000000  102.666667  6.933333
     1972    338.000000   90.500000  7.500000
     1973    980.600000  101.600000  7.280000
     1974    116.428571   98.857143  6.771429
     1975    246.000000   82.111111  6.611111
     1976    289.909091   74.363636  6.336364
     1977    220.125000   83.250000  6.525000
     1978    261.800000   75.800000  6.800000
     1979    492.937500   88.000000  6.956250
     1980    244.444444   84.333333  7.222222
     1981    414.300000   81.700000  7.140000
     1982    885.571429   86.714286  6.942857
     1983   2291.900000  100.100000  7.750000
     1984    955.800000   91.000000  7.410000
     1985   2601.000000   82.285714  7.314286
     1986   1148.000000   79.181818  7.418182
     1987   4032.750000   92.125000  7.550000
     1988   3576.222222   75.666667  6.877778
     1989   5792.428571   93.428571  7.857143
     1990   1476.857143   94.285714  6.900000
     1991  24040.400000  113.400000  7.100000
     1992   4683.250000   99.875000  7.600000
     1993  21047.600000  106.800000  6.840000
...                 ...         ...       ...
韩国   1986    109.000000   60.000000  6.600000
     1987    387.000000   90.000000  5.700000
     1989    203.000000  149.500000  7.950000
     1990     66.000000  132.500000  7.350000
     1991    140.500000   60.000000  6.100000
     1992    112.500000  114.000000  6.850000
     1993    267.600000  111.800000  7.460000
     1994     65.000000  111.666667  6.666667
     1995    330.250000   91.000000  6.600000
     1996    512.428571   97.285714  6.985714
     1997    789.071429   99.071429  6.778571
     1998   3921.411765   95.588235  6.852941
     1999   2329.260870   94.478261  6.834783
     2000   4688.400000   94.433333  6.383333
     2001  10745.363636   99.484848  6.709091
     2002   5189.895833  102.145833  6.641667
     2003  15509.218182   98.763636  6.814545
     2004   9514.093750  102.750000  6.814062
     2005   6427.630137   98.493151  6.830137
     2006   7454.670455  103.284091  6.737500
     2007   3769.526316  101.013158  6.625000
     2008   6738.689189  100.648649  6.731081
     2009   4688.767123   96.863014  6.505479
     2010   8615.433333   99.344444  6.473333
     2011   8949.400000  100.011111  6.365556
     2012   5762.537736  100.669811  6.064151
     2013  10189.036036   96.504505  6.098198
     2014   3776.266667   98.666667  5.650833
     2015   3209.247706  100.266055  5.423853
     2016   1739.850000  106.100000  5.730000

[1584 rows x 3 columns]

获得每个地区，每一年的电影的评分的均值

In [178]:
means=df['评分'].groupby([df['产地'],df['年代']]).mean()
means

产地    年代  
中国台湾  1963    6.400000
      1965    6.800000
      1966    7.900000
      1967    8.000000
      1968    7.400000
      1969    7.400000
      1970    7.266667
      1971    6.933333
      1972    7.500000
      1973    7.280000
      1974    6.771429
      1975    6.611111
      1976    6.336364
      1977    6.525000
      1978    6.800000
      1979    6.956250
      1980    7.222222
      1981    7.140000
      1982    6.942857
      1983    7.750000
      1984    7.410000
      1985    7.314286
      1986    7.418182
      1987    7.550000
      1988    6.877778
      1989    7.857143
      1990    6.900000
      1991    7.100000
      1992    7.600000
      1993    6.840000
                ...   
韩国    1986    6.600000
      1987    5.700000
      1989    7.950000
      1990    7.350000
      1991    6.100000
      1992    6.850000
      1993    7.460000
      1994    6.666667
      1995    6.600000
      1996    6.985714
      1997    6.778571
      1998    6.852941


series通过unstack方法转化为dataframe

In [179]:
means.unstack() #.T

年代,1888,1890,1892,1894,1895,1896,1897,1898,1899,1900,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
产地,,,,,,,,,,,,,,,,,,,,,
中国台湾,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.968421,7.420000,7.100000,7.053846,7.231818,6.556098,7.076471,6.522222,6.576000,NaN
中国大陆,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.605691,6.589726,6.536628,6.357831,5.824606,5.727187,5.316667,4.963757,4.969189,4.712000
中国香港,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.085106,6.476786,6.442553,6.245455,5.971053,6.110526,6.105714,5.616667,5.589189,5.390909
丹麦,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.575000,7.050000,7.118182,7.362500,7.016667,7.418750,6.555556,7.120000,7.166667,7.000000
俄罗斯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.637037,6.741667,6.278571,6.858824,7.040000,6.572727,6.875000,7.175000,7.342857,NaN
其他,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.100000,7.149600,6.789091,6.866038,6.792000,6.591743,6.853571,6.596250,6.735714,7.225000
加拿大,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.440678,6.708197,6.544186,6.500000,6.100000,6.425000,6.018182,5.921739,6.018750,6.200000
印度,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.050000,6.642857,6.728000,6.812903,6.610000,6.502632,6.400000,6.374194,6.736364,6.900000
墨西哥,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.233333,7.283333,7.000000,6.333333,6.550000,7.100000,6.983333,7.250000,6.500000,NaN


会产生缺失值

## 离散化处理
在实际的数据分析项目中，对有的数据属性，我们往往并不关注数据的绝对取值，只关注它所处的区间或者等级，比如

可以把评分9分以上的电影定义为A，把评分7到9分之间的定为B，5到7分定义为C，3到5分定义为D，小于3分定义为E

离散化也可称为分组、区间化。

Pandas提供了方便的函数：cut()，用法：

pd.cut(x,bins,right=True,labels=None,retbins=False,precision=3，include_lowest=False)

参数解释：

x:需要离散化的数组、Series、DataFrame对象

bins：分组的依据，区间的划分，可以是数字、序列，

right右端点，默认包括；include。。默认不包括左端

labels:各等级的名称

In [48]:
df['评分']

0        9.6
1        9.5
2        9.5
3        9.4
4        9.4
5        9.4
6        9.4
7        9.4
8        9.4
9        9.4
10       9.3
11       9.3
12       9.3
13       9.3
14       9.3
15       9.3
16       9.3
17       9.3
18       9.2
19       9.2
20       9.2
21       9.2
22       9.2
23       9.2
24       9.2
25       9.2
26       9.2
27       9.2
28       9.2
29       9.2
        ... 
38706    7.5
38707    8.4
38708    8.1
38709    6.8
38710    6.1
38711    8.0
38712    7.7
38713    6.9
38714    7.0
38715    6.2
38716    8.1
38717    8.5
38718    9.1
38719    8.1
38720    8.2
38721    8.2
38722    7.8
38723    8.0
38724    7.4
38725    8.7
38726    8.2
38727    7.4
38728    8.7
38729    8.1
38730    8.1
38731    8.6
38732    7.6
38733    7.1
38734    8.0
38735    6.6
Name: 评分, Length: 38736, dtype: float64

In [57]:
df['评分等级']=pd.cut(df['评分'],[0,3,5,7,9,10],labels=['E','D','C','B','A'])
df

,名字,投票人数,类型,产地,上映时间,时长,年代,评分,首映地点,分级,热门程度,评分等级
0,肖申克的救赎,692795,剧情/犯罪,美国,1994-09-10 00:00:00,142,1994,9.6,多伦多电影节,A,A,A
1,控方证人,42995,剧情/悬疑/犯罪,美国,1957-12-17 00:00:00,116,1957,9.5,美国,A,A,A
2,美丽人生,327855,剧情/喜剧/爱情,意大利,1997-12-20 00:00:00,116,1997,9.5,意大利,A,A,A
3,阿甘正传,580897,剧情/爱情,美国,1994-06-23 00:00:00,142,1994,9.4,洛杉矶首映,A,A,A
4,霸王别姬,478523,剧情/爱情/同性,中国大陆,1993-01-01 00:00:00,171,1993,9.4,香港,A,A,A
5,泰坦尼克号,157074,剧情/爱情/灾难,美国,2012-04-10 00:00:00,194,2012,9.4,中国大陆,A,A,A
6,辛德勒的名单,306904,剧情/历史/战争,美国,1993-11-30 00:00:00,195,1993,9.4,华盛顿首映,A,A,A
7,新世纪福音战士剧场版：Air/真心为你 新世紀エヴァンゲリオン劇場版 Ai,24355,剧情/动作/科幻/动画/奇幻,日本,1997-07-19 00:00:00,87,1997,9.4,日本,A,A,A
8,银魂完结篇：直到永远的万事屋 劇場版 銀魂 完結篇 万事屋よ,21513,剧情/动画,日本,2013-07-06 00:00:00,110,2013,9.4,日本,A,A,A
9,这个杀手不太冷,662552,剧情/动作/犯罪,法国,1994-09-14 00:00:00,133,1994,9.4,法国,A,A,A


同样的，我们可以根据投票人数来刻画电影的热门

投票越多的热门程度越高

In [50]:
df['投票人数'].max()

692795

In [51]:
#计算一个数组的任意百分比分位数，此处的百分位是从小到大排列，用np.percentile(..,..)
a = range(1,101)
#求取a数列第90%分位的数值
np.percentile(a, 90)

90.10000000000001

In [52]:
bins=np.percentile(df['投票人数'],[0,20,40,60,80,100])  
bins

array([-1.18000e+02,  7.80000e+01,  1.98000e+02,  6.17000e+02,
        2.74600e+03,  6.92795e+05])

In [55]:
df['热门程度']=pd.cut(df['投票人数'],bins,labels=['E','D','C','B','A'])
df

,名字,投票人数,类型,产地,上映时间,时长,年代,评分,首映地点,分级,热门程度
0,肖申克的救赎,692795,剧情/犯罪,美国,1994-09-10 00:00:00,142,1994,9.6,多伦多电影节,A,A
1,控方证人,42995,剧情/悬疑/犯罪,美国,1957-12-17 00:00:00,116,1957,9.5,美国,A,A
2,美丽人生,327855,剧情/喜剧/爱情,意大利,1997-12-20 00:00:00,116,1997,9.5,意大利,A,A
3,阿甘正传,580897,剧情/爱情,美国,1994-06-23 00:00:00,142,1994,9.4,洛杉矶首映,A,A
4,霸王别姬,478523,剧情/爱情/同性,中国大陆,1993-01-01 00:00:00,171,1993,9.4,香港,A,A
5,泰坦尼克号,157074,剧情/爱情/灾难,美国,2012-04-10 00:00:00,194,2012,9.4,中国大陆,A,A
6,辛德勒的名单,306904,剧情/历史/战争,美国,1993-11-30 00:00:00,195,1993,9.4,华盛顿首映,A,A
7,新世纪福音战士剧场版：Air/真心为你 新世紀エヴァンゲリオン劇場版 Ai,24355,剧情/动作/科幻/动画/奇幻,日本,1997-07-19 00:00:00,87,1997,9.4,日本,A,A
8,银魂完结篇：直到永远的万事屋 劇場版 銀魂 完結篇 万事屋よ,21513,剧情/动画,日本,2013-07-06 00:00:00,110,2013,9.4,日本,A,A
9,这个杀手不太冷,662552,剧情/动作/犯罪,法国,1994-09-14 00:00:00,133,1994,9.4,法国,A,A


大烂片集合：投票人数很多，评分很低

In [58]:
df[(df.热门程度=='A') & (df.评分等级=='E')]

,名字,投票人数,类型,产地,上映时间,时长,年代,评分,首映地点,分级,热门程度,评分等级
655,B区,5187,剧情/惊悚/恐怖,中国大陆,2011-06-03 00:00:00,89,2011,2.3,中国大陆,E,A,E
4376,怖偶,4867,悬疑/惊悚,中国大陆,2014-05-07 00:00:00,88,2014,2.8,中国大陆,E,A,E
5413,床下有人,4309,悬疑/惊悚,中国大陆,2011-10-14 00:00:00,100,2011,2.8,中国大陆,E,A,E
6802,帝国秘符,4351,动作/冒险,中国大陆,2013-09-18 00:00:00,93,2013,3.0,中国大陆,E,A,E
8232,飞天,4764,剧情,中国大陆,2011-07-01 00:00:00,115,2011,2.9,中国大陆,E,A,E
8406,分手达人,3937,喜剧/爱情,中国大陆,2014-06-06 00:00:00,90,2014,2.7,中国大陆,E,A,E
9601,孤岛惊魂,2982,悬疑/惊悚/恐怖,中国大陆,2013-01-26 00:00:00,93,2012,2.8,中国大陆,E,A,E
10513,海天盛宴·韦口,3788,情色,中国大陆,2013-10-12 00:00:00,88,2013,2.9,网络,E,A,E
16799,孪生密码,6390,动作/悬疑,中国大陆,2013-11-08 00:00:00,96,2013,2.9,中国大陆,E,A,E
21537,拳皇,6329,动作/科幻/冒险,日本,2012-10-12 00:00:00,93,2010,3.0,中国大陆,E,A,E


冷门高分电影

In [59]:
df[(df.热门程度=='E') & (df.评分等级=='A')]

,名字,投票人数,类型,产地,上映时间,时长,年代,评分,首映地点,分级,热门程度,评分等级
456,NaN,45,音乐,美国,2015-12-14 00:00:00,60,2015,9.3,美国,A,E,A
587,BBC喜剧音,38,喜剧/音乐/歌舞,英国,2011-08-13 00:00:00,95,2011,9.3,美国,A,E,A
642,NaN,38,纪录片/音乐,美国,2003-02-03 00:00:00,55,2003,9.2,美国,A,E,A
686,NaN,63,音乐/舞台艺术,英国,2014-05-18 00:00:00,49,2014,9.5,美国,A,E,A
698,NaN,52,纪录片,英国,2008-07-25 00:00:00,40,2008,9.8,美国,A,E,A
844,NaN,69,纪录片/运动,美国,2007-12-07 00:00:00,129,2007,9.1,美国,A,E,A
925,NaN,33,纪录片,澳大利亚,2012-04-10 00:00:00,78,2012,9.5,美国,A,E,A
979,JOJO的奇妙冒险 特别见面会 Walk Like Crusade,36,纪录片,日本,2014-10-26 00:00:00,137,2014,9.3,美国,A,E,A
1081,NaN,58,音乐,美国,2002-10-22 00:00:00,60,2002,9.2,美国,A,E,A
1107,NaN,60,音乐,美国,1905-06-23 00:00:00,190,2001,9.3,美国,A,E,A


将处理后的数据进行保存

In [60]:
df.to_excel('movie_data3.xlsx')

## 合并数据集
（1） append

先把数据集拆分为多个，再进行合并

In [61]:
import pandas as pd
df=pd.read_excel('movie_data3.xlsx')

In [62]:
df_usa=df[df.产地=='美国']
df_usa

,Unnamed: 0,名字,投票人数,类型,产地,上映时间,时长,年代,评分,首映地点,分级,热门程度,评分等级
0,0,肖申克的救赎,692795,剧情/犯罪,美国,1994-09-10 00:00:00,142,1994,9.6,多伦多电影节,A,A,A
1,1,控方证人,42995,剧情/悬疑/犯罪,美国,1957-12-17 00:00:00,116,1957,9.5,美国,A,A,A
3,3,阿甘正传,580897,剧情/爱情,美国,1994-06-23 00:00:00,142,1994,9.4,洛杉矶首映,A,A,A
5,5,泰坦尼克号,157074,剧情/爱情/灾难,美国,2012-04-10 00:00:00,194,2012,9.4,中国大陆,A,A,A
6,6,辛德勒的名单,306904,剧情/历史/战争,美国,1993-11-30 00:00:00,195,1993,9.4,华盛顿首映,A,A,A
11,11,疯狂动物城,284652,喜剧/动作/动画/冒险,美国,2016-03-04 00:00:00,109,2016,9.3,中国大陆/美国,A,A,A
13,13,海豚湾,159302,纪录片,美国,2009-07-31 00:00:00,92,2009,9.3,美国,A,A,A
15,15,机器人总动员,421734,喜剧/爱情/科幻/动画/冒险,美国,2008-06-27 00:00:00,98,2008,9.3,美国,A,A,A
16,16,十二怒汉,134949,剧情,美国,1957-04-01 00:00:00,96,1957,9.3,美国,A,A,A
17,17,旅行到宇宙边缘,10044,纪录片,美国,2008-11-07 00:00:00,90,2008,9.3,美国,A,A,A


In [63]:
df_china=df[df.产地=='中国大陆']
df_china

,Unnamed: 0,名字,投票人数,类型,产地,上映时间,时长,年代,评分,首映地点,分级,热门程度,评分等级
4,4,霸王别姬,478523,剧情/爱情/同性,中国大陆,1993-01-01 00:00:00,171,1993,9.4,香港,A,A,A
21,21,大闹天宫,74881,动画/奇幻,中国大陆,1905-05-14 00:00:00,114,1961,9.2,上集,A,A,A
29,29,穹顶之下,51113,纪录片,中国大陆,2015-02-28 00:00:00,104,2015,9.2,中国大陆,A,A,A
38,38,茶馆,10678,剧情/历史,中国大陆,1905-06-04 00:00:00,118,1982,9.2,美国,A,A,A
45,45,山水情,10781,动画/短片,中国大陆,1905-06-10 00:00:00,19,1988,9.2,美国,A,A,A
49,49,“人权卫士”的人权纪录,109,纪录片,中国大陆,2016-03-13 00:00:00,45,2016,5.6,中国大陆,C,D,C
62,62,0.5,2825,爱情,中国大陆,2014-09-12 00:00:00,85,2014,4.4,中国大陆,D,A,D
67,67,鬼子来了,180738,剧情/战争,中国大陆,2000-05-12 00:00:00,139,2000,9.1,戛纳电影节,A,A,A
119,119,11度青春之《李雷和韩梅,9583,短片,中国大陆,2010-10-21 00:00:00,60,2008,5.7,美国,C,A,C
132,132,12只蚊子和,1231,动画/短片,中国大陆,1905-06-14 00:00:00,6,1992,8.3,美国,B,B,B


In [64]:
df_china.append(df_usa)

,Unnamed: 0,名字,投票人数,类型,产地,上映时间,时长,年代,评分,首映地点,分级,热门程度,评分等级
4,4,霸王别姬,478523,剧情/爱情/同性,中国大陆,1993-01-01 00:00:00,171,1993,9.4,香港,A,A,A
21,21,大闹天宫,74881,动画/奇幻,中国大陆,1905-05-14 00:00:00,114,1961,9.2,上集,A,A,A
29,29,穹顶之下,51113,纪录片,中国大陆,2015-02-28 00:00:00,104,2015,9.2,中国大陆,A,A,A
38,38,茶馆,10678,剧情/历史,中国大陆,1905-06-04 00:00:00,118,1982,9.2,美国,A,A,A
45,45,山水情,10781,动画/短片,中国大陆,1905-06-10 00:00:00,19,1988,9.2,美国,A,A,A
49,49,“人权卫士”的人权纪录,109,纪录片,中国大陆,2016-03-13 00:00:00,45,2016,5.6,中国大陆,C,D,C
62,62,0.5,2825,爱情,中国大陆,2014-09-12 00:00:00,85,2014,4.4,中国大陆,D,A,D
67,67,鬼子来了,180738,剧情/战争,中国大陆,2000-05-12 00:00:00,139,2000,9.1,戛纳电影节,A,A,A
119,119,11度青春之《李雷和韩梅,9583,短片,中国大陆,2010-10-21 00:00:00,60,2008,5.7,美国,C,A,C
132,132,12只蚊子和,1231,动画/短片,中国大陆,1905-06-14 00:00:00,6,1992,8.3,美国,B,B,B


In [196]:
df_china

,Unnamed: 0,产地,年代,名字,投票人数,类型,上映时间,时长,评分,首映地点,评分等级,热门程度
4,4,中国大陆,1993,霸王别姬,478523,剧情/爱情/同性,1993-01-01 00:00:00,171,9.4,香港,A,A
21,21,中国大陆,1961,大闹天宫,74881,动画/奇幻,1905-05-14 00:00:00,114,9.2,上集,A,A
29,29,中国大陆,2015,穹顶之下,51113,纪录片,2015-02-28 00:00:00,104,9.2,中国大陆,A,A
38,38,中国大陆,1982,茶馆,10678,剧情/历史,1905-06-04 00:00:00,118,9.2,美国,A,A
45,45,中国大陆,1988,山水情,10781,动画/短片,1905-06-10 00:00:00,19,9.2,美国,A,A
49,49,中国大陆,2016,“人权卫士”的人权纪录,109,纪录片,2016-03-13 00:00:00,45,5.6,中国大陆,C,D
62,62,中国大陆,2014,0.5,2825,爱情,2014-09-12 00:00:00,85,4.4,中国大陆,D,A
67,67,中国大陆,2000,鬼子来了,180738,剧情/战争,2000-05-12 00:00:00,139,9.1,戛纳电影节,A,A
119,119,中国大陆,2008,11度青春之《李雷和韩梅,9583,短片,2010-10-21 00:00:00,60,5.7,美国,C,A
132,132,中国大陆,1992,12只蚊子和,1231,动画/短片,1905-06-14 00:00:00,6,8.3,美国,B,B


将这两个数据集进行合并了

（2）merge 横向拼接

pd.merge(left,right,how='inner',on=None,left_on=None,right_on=None,
         
         left_index=False,right_index=False,sort=True,
         
         suffixes=('_x','_y'),copy=True,indicator=False)

left:对象

right:另一个对象

how:‘left’‘right’‘outer’‘inner’，默认为'inner'。

on:连接的列（名称）。必须在左、右对象中找到。如果不能通过left_index和right_index,将推断DataFrames中的列的交叉点为连接键

left_on：左边的键列。

right_on:右边的键列。

left_index:如果为True，则使用左边的索引（行标签）作为连接键

right_index：如果为True，则使用右边的索引（行标签）作为连接键

sort：通过连接键按字典顺序对结果进行排序。设置为False将在许多情况下极大地提高性能。

suffixes:多个对象中有相同列名称时，生成后缀加以区分，默认值为('_x','_y')。
   

我们选取6部热门电影

In [65]:
df1=df.loc[:5]
df1

,Unnamed: 0,名字,投票人数,类型,产地,上映时间,时长,年代,评分,首映地点,分级,热门程度,评分等级
0,0,肖申克的救赎,692795,剧情/犯罪,美国,1994-09-10 00:00:00,142,1994,9.6,多伦多电影节,A,A,A
1,1,控方证人,42995,剧情/悬疑/犯罪,美国,1957-12-17 00:00:00,116,1957,9.5,美国,A,A,A
2,2,美丽人生,327855,剧情/喜剧/爱情,意大利,1997-12-20 00:00:00,116,1997,9.5,意大利,A,A,A
3,3,阿甘正传,580897,剧情/爱情,美国,1994-06-23 00:00:00,142,1994,9.4,洛杉矶首映,A,A,A
4,4,霸王别姬,478523,剧情/爱情/同性,中国大陆,1993-01-01 00:00:00,171,1993,9.4,香港,A,A,A
5,5,泰坦尼克号,157074,剧情/爱情/灾难,美国,2012-04-10 00:00:00,194,2012,9.4,中国大陆,A,A,A


In [66]:
df2=df.loc[:5][['名字','产地']]
df2['票房']=[45645,343445,45454,46565,45546,787]
df2

,名字,产地,票房
0,肖申克的救赎,美国,45645
1,控方证人,美国,343445
2,美丽人生,意大利,45454
3,阿甘正传,美国,46565
4,霸王别姬,中国大陆,45546
5,泰坦尼克号,美国,787


In [67]:
df2=df2.sample(frac=1)  #记录打乱
df2

,名字,产地,票房
5,泰坦尼克号,美国,787
0,肖申克的救赎,美国,45645
3,阿甘正传,美国,46565
4,霸王别姬,中国大陆,45546
2,美丽人生,意大利,45454
1,控方证人,美国,343445


In [68]:
df2.index=range(len(df2))
df2

,名字,产地,票房
0,泰坦尼克号,美国,787
1,肖申克的救赎,美国,45645
2,阿甘正传,美国,46565
3,霸王别姬,中国大陆,45546
4,美丽人生,意大利,45454
5,控方证人,美国,343445


现在，需要把df1和df2合并

我们发现,df2有票房数据，df1有评分等其他信息，

由于样本的顺序不一致，因此不能采取直接复制的方法。

In [69]:
pd.merge(df1,df2,how='inner',on='名字')

,Unnamed: 0,名字,投票人数,类型,产地_x,上映时间,时长,年代,评分,首映地点,分级,热门程度,评分等级,产地_y,票房
0,0,肖申克的救赎,692795,剧情/犯罪,美国,1994-09-10 00:00:00,142,1994,9.6,多伦多电影节,A,A,A,美国,45645
1,1,控方证人,42995,剧情/悬疑/犯罪,美国,1957-12-17 00:00:00,116,1957,9.5,美国,A,A,A,美国,343445
2,2,美丽人生,327855,剧情/喜剧/爱情,意大利,1997-12-20 00:00:00,116,1997,9.5,意大利,A,A,A,意大利,45454
3,3,阿甘正传,580897,剧情/爱情,美国,1994-06-23 00:00:00,142,1994,9.4,洛杉矶首映,A,A,A,美国,46565
4,4,霸王别姬,478523,剧情/爱情/同性,中国大陆,1993-01-01 00:00:00,171,1993,9.4,香港,A,A,A,中国大陆,45546
5,5,泰坦尼克号,157074,剧情/爱情/灾难,美国,2012-04-10 00:00:00,194,2012,9.4,中国大陆,A,A,A,美国,787


由于两个数据集都存在产地，因此合并后会有两个产地信息

（3）concat
将多个数据集进行批量合并

In [70]:
df1=df[:10]
df2=df[100:110]
df3=df[300:310]
df2

,Unnamed: 0,名字,投票人数,类型,产地,上映时间,时长,年代,评分,首映地点,分级,热门程度,评分等级
100,100,101,146,喜剧/爱情,韩国,1993-06-19 00:00:00,112,1993,7.4,韩国,B,D,B
101,101,10,186,喜剧,英国,1995-01-25 00:00:00,101,1995,7.4,美国,B,D,B
102,102,素媛,114819,剧情/家庭,韩国,2013-10-02 00:00:00,123,2013,9.1,韩国,A,A,A
103,103,101忠狗续集：伦敦,924,喜剧/动画/家庭,美国,2003-01-21 00:00:00,70,2003,7.5,美国,B,B,B
104,104,10,9514,喜剧/家庭,美国,2000-09-22 00:00:00,100,2000,7.0,美国,C,A,C
105,105,10,601,剧情,韩国,2014-04-24 00:00:00,93,2013,7.2,美国,B,C,B
106,106,10件或,1770,剧情/喜剧/爱情,美国,2006-12-01 00:00:00,82,2006,7.7,美国,B,B,B
107,107,10年,1531,喜剧/同性,美国,2015-06-02 00:00:00,90,2014,6.9,美国,C,B,C
108,108,11·25自决之日 三岛由纪夫与年轻人们 11・25自決の,149,剧情,日本,2012-06-02 00:00:00,119,2012,5.6,日本,C,D,C
109,109,泰坦尼克号,535491,剧情/爱情/灾难,美国,1998-04-03 00:00:00,194,1997,9.1,中国大陆,A,A,A


In [71]:
dff=pd.concat([df1,df2,df3],axis=0) #默认增加行
dff

,Unnamed: 0,名字,投票人数,类型,产地,上映时间,时长,年代,评分,首映地点,分级,热门程度,评分等级
0,0,肖申克的救赎,692795,剧情/犯罪,美国,1994-09-10 00:00:00,142,1994,9.6,多伦多电影节,A,A,A
1,1,控方证人,42995,剧情/悬疑/犯罪,美国,1957-12-17 00:00:00,116,1957,9.5,美国,A,A,A
2,2,美丽人生,327855,剧情/喜剧/爱情,意大利,1997-12-20 00:00:00,116,1997,9.5,意大利,A,A,A
3,3,阿甘正传,580897,剧情/爱情,美国,1994-06-23 00:00:00,142,1994,9.4,洛杉矶首映,A,A,A
4,4,霸王别姬,478523,剧情/爱情/同性,中国大陆,1993-01-01 00:00:00,171,1993,9.4,香港,A,A,A
5,5,泰坦尼克号,157074,剧情/爱情/灾难,美国,2012-04-10 00:00:00,194,2012,9.4,中国大陆,A,A,A
6,6,辛德勒的名单,306904,剧情/历史/战争,美国,1993-11-30 00:00:00,195,1993,9.4,华盛顿首映,A,A,A
7,7,新世纪福音战士剧场版：Air/真心为你 新世紀エヴァンゲリオン劇場版 Ai,24355,剧情/动作/科幻/动画/奇幻,日本,1997-07-19 00:00:00,87,1997,9.4,日本,A,A,A
8,8,银魂完结篇：直到永远的万事屋 劇場版 銀魂 完結篇 万事屋よ,21513,剧情/动画,日本,2013-07-06 00:00:00,110,2013,9.4,日本,A,A,A
9,9,这个杀手不太冷,662552,剧情/动作/犯罪,法国,1994-09-14 00:00:00,133,1994,9.4,法国,A,A,A
